<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TensorFlow/Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to TensorFlow 2

In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")
import time
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
x_train = {
    "type": [0, 1, 1],
    "size": ["small", "small", "medium"],
    "weight": [2.7, 1.8, 1.6]
}
y_train = [1, 1, 0]
ds_train = tf.data.Dataset.from_tensor_slices(tensors=(x_train, y_train))
ds_train = ds_train.map(map_func=lambda x, y:
    ({k: tf.expand_dims(input=v, axis=-1) for k, v in x.items()}, y))\
    .batch(batch_size=1)
                        
inputs = {}
inputs["type"] = tf.keras.Input(name="type", shape=(), dtype=tf.int64)
inputs["size"] = tf.keras.Input(name="size", shape=(), dtype=tf.string)
inputs["weight"] = tf.keras.Input(name="weight", shape=(), dtype=tf.float32)

outputs = {}
outputs["type"] = tf.keras.layers.CategoryEncoding(
    num_tokens=3, output_mode="one_hot")(inputs["type"])
outputs["size"] = tf.keras.layers.StringLookup(
    vocabulary=["small", "medium", "large"])(inputs["size"])
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(ds_train.map(lambda x, y: x["weight"]))
outputs["weight"] =  normalizer(inputs["weight"])

preprocessing_model = tf.keras.Model(inputs=inputs, outputs=outputs)

ds_train = ds_train.map(lambda x, y: (preprocessing_model(x), y),
    num_parallel_calls=tf.data.AUTOTUNE)
next(iter(ds_train.take(count=1)))

({'type': <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[1., 0., 0.]], dtype=float32)>,
  'size': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
  'weight': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.3934661]], dtype=float32)>},
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>)

In [4]:
inputs = {}
inputs["type"] = tf.keras.Input(name="type", shape=(3,), dtype=tf.float32)
inputs["size"] = tf.keras.Input(name="size", shape=(), dtype=tf.int64)
inputs["weight"] = tf.keras.Input(name="weight", shape=(1,), dtype=tf.float32)

embedding = tf.keras.layers.Embedding(input_dim=3, output_dim=4)

h = tf.keras.layers.Concatenate()([
    inputs["type"],
    embedding(inputs["size"]),
    inputs["weight"]
])
outputs = tf.keras.layers.Dense(units=1)(h)
training_model = tf.keras.Model(inputs=inputs, outputs=outputs)

training_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))
training_model.fit(x=ds_train)

I0000 00:00:1718463304.148049    1597 service.cc:145] XLA service 0x7ac8f8005ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718463304.148128    1597 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1660, Compute Capability 7.5


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5299


I0000 00:00:1718463304.732951    1597 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [8]:
inputs = preprocessing_model.input
outputs = training_model(preprocessing_model(inputs))
inference_model = tf.keras.Model(inputs=inputs, outputs=outputs)
inference_model.save("model.keras")

x_test = {
    "type": [0],
    "size": ["foo"],
    "weight": [-0.7]
}
ds_test = tf.data.Dataset.from_tensor_slices(tensors=x_test)
ds_test = ds_test.map(map_func=lambda x:
    {k: tf.expand_dims(input=v, axis=-1) for k, v in x.items()})\
    .batch(batch_size=1)

inference_model = tf.keras.models.load_model("model.keras")
inference_model.predict(ds_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 911ms/step


array([[0.5399132]], dtype=float32)

In [2]:
dataset_url = "http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip"
csv_file = "datasets/petfinder-mini/petfinder-mini.csv"

tf.keras.utils.get_file(fname="petfinder_mini.zip", origin=dataset_url, extract=True,
    cache_dir=".")
df = pd.read_csv(csv_file)
df.drop(inplace=True, columns=["Description"])
df.info()
df.head()

1668792/1668792 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           11537 non-null  object
 1   Age            11537 non-null  int64 
 2   Breed1         11537 non-null  object
 3   Gender         11537 non-null  object
 4   Color1         11537 non-null  object
 5   Color2         11537 non-null  object
 6   MaturitySize   11537 non-null  object
 7   FurLength      11537 non-null  object
 8   Vaccinated     11537 non-null  object
 9   Sterilized     11537 non-null  object
 10  Health         11537 non-null  object
 11  Fee            11537 non-null  int64 
 12  PhotoAmt       11537 non-null  int64 
 13  AdoptionSpeed  11537 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 1.2+ MB


,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,3,2


In [ ]:
def df_to_ds(dataframe, batch_size, shuffle=False):
    df = dataframe.copy()
    y = df.pop("label")
    x = {k: v.values[:, tf.newaxis] for k, v in df.items()}
    ds = tf.data.Dataset.from_tensor_slices(tensors=(x, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size=batch_size)
    ds = ds.prefetch(buffer_size=batch_size)
    return ds

df_train, df_test = model_selection.train_test_split(
    df, test_size=0.1, random_state=42)

ds_train = df_to_ds(dataframe=df_train, batch_size=256, shuffle=True)
ds_test = df_to_ds(dataframe=df_test, batch_size=32)
ds_test.element_spec

In [ ]:
inputs = []
features = []

for h in ["Fee", "PhotoAmt"]:
    col = tf.keras.Input(shape=(), name=h, dtype=tf.int64)
    inputs.append(col)
    x = ds_train.map(map_func=lambda x, y: x[h])
    normalizer = tf.keras.layers.Normalization(axis=None)
    normalizer.adapt(x)
    feature = normalizer(col)
    features.append(feature)

for h in ["Type", "Breed1", "Gender", "Color1", "Color2", "MaturitySize", "FurLength",
        "Vaccinated", "Sterilized", "Health"]:
    col = tf.keras.Input(shape=(), name=h, dtype=tf.string)
    inputs.append(col)
    x = ds_train.map(map_func=lambda x, y: x[h])
    index = tf.keras.layers.StringLookup(max_tokens=5)
    index.adapt(x)
    feature = tf.keras.layers.CategoryEncoding(
        num_tokens=index.vocabulary_size())
    features.append(feature)

for h in ["Age"]:
    col = tf.keras.Input(shape=(), name=h, dtype=tf.int64)
    inputs.append(col)
    x = ds_train.map(map_func=lambda x, y: x[h])
    index = tf.keras.layers.IntegerLookup(max_tokens=5)
    index.adapt(x)
    feature = tf.keras.layers.CategoryEncoding(
        num_tokens=index.vocabulary_size())
    features.append(feature)

In [ ]:
outputs = tf.keras.layers.concatenate(features)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

### UCI Bank Marketing Dataset

The `Bank Marketing` dataset is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed. [here](https://archive.ics.uci.edu/dataset/222/bank+marketing).

In [ ]:
df_bank = pd.read_csv("data/bank/bank-full.csv", delimiter=";")
df_bank.info()
df_bank.tail()

In [ ]:
ds_bank = tf.data.Dataset.from_tensor_slices(tensors=dict(df_bank))
ds_bank.element_spec

In [ ]:
inputs = {
    'age': tf.keras.Input(shape=(1,), dtype=tf.int64, name='age')
}

age = tf.keras.layers.Discretization(
    bin_boundaries=[10, 20, 30, 40, 50, 60, 70, 80, 90])(inputs['age'])
age = tf.keras.layers.CategoryEncoding(num_tokens=10, output_mode='one_hot')(age)

outputs = {
    'age': age
}

preprocessing_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
ds01 = ds_bank.batch(1)
ds01 = ds01.map(lambda x: preprocessing_model(x))
next(iter(ds01))

In [ ]:
df_train, df_test = model_selection.train_test_split(
    df_bank, test_size=0.1, random_state=42)

dict_train = dict(df_train)
ds_train = tf.data.Dataset.from_tensor_slices(tensors={
    k: dict_train[k] for k in dict_train})

dict_test = dict(df_test)
ds_test = tf.data.Dataset.from_tensor_slices(tensors={
    k: dict_test[k] for k in dict_test})
ds_test.element_spec

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

batch_size = 64

ds_train = tf.data.Dataset.from_tensor_slices(tensors=(x_train, y_train))
ds_train = ds_train.map(map_func=lambda x, y: (
    tf.cast(x=x, dtype=tf.float32)/255., y))
ds_train = ds_train.map(map_func=lambda x, y: (
    tf.reshape(x, (784,)), y))
ds_train = ds_train.shuffle(buffer_size=len(ds_train))\
    .batch(batch_size=batch_size)

ds_test = tf.data.Dataset.from_tensor_slices(tensors=(x_test, y_test))
ds_test = ds_test.map(map_func=lambda x, y:
    (tf.cast(x=x, dtype=tf.float32)/255., y))
ds_test = ds_test.map(map_func=lambda x, y: (
    tf.reshape(x, (784,)), y))
ds_test = ds_test.batch(batch_size=batch_size)
print("Element spec:", ds_test.element_spec)

In [ ]:
def get_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    h = tf.keras.layers.Dense(units=64,
        activation=tf.keras.activations.relu,
        kernel_regularizer=tf.keras.regularizers.L2(l2=1e-5))(inputs)
    h = tf.keras.layers.Dropout(rate=0.2)(h)
    outputs = tf.keras.layers.Dense(units=num_classes,
        activation=tf.keras.activations.softmax)(h)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

model = get_model(input_shape=(784,), num_classes=10)
model.summary()

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")]
)

@tf.function
def train_step(x, y):
    logs = {}
    with tf.GradientTape() as tape:
        y_pred = model(inputs=x, training=True)
        loss = model.compute_loss(y=y, y_pred=y_pred)

    trainable_vars = model.trainable_variables
    grads = tape.gradient(target=loss,
        sources=trainable_vars)
    model.optimizer.apply_gradients(grads_and_vars=zip(
        grads, trainable_vars))

    for metric in model.metrics:
        if metric.name=="loss":
            metric.update_state(values=loss)
        else:
            metric.update_state(y_true=y, y_pred=y_pred)

    metrics = [m.result() for m in model.metrics]
    logs = {f"train_{k}": v for k, v in metrics[1].items()}
    logs["train_loss"] = metrics[0]

    return logs

@tf.function
def test_step(x, y):
    y_pred = model(inputs=x, training=False)
    loss = model.compute_loss(y=y, y_pred=y_pred)

    for metric in model.metrics:
        if metric.name=="loss":
            metric.update_state(values=loss)
        else:
            metric.update_state(y_true=y, y_pred=y_pred)

    metrics = [m.result() for m in model.metrics]
    logs = {f"test_{k}": v for k, v in metrics[1].items()}
    logs["test_loss"] = metrics[0]

    return logs

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="test_loss",
    patience=1)
callbacks = tf.keras.callbacks.CallbackList(model=model,
    callbacks=[reduce_lr], add_history=True)

In [ ]:
num_epochs = 10
start_time = time.time()
history = {
    "train_loss": [], "test_loss": [],
    "train_accuracy": [], "test_accuracy": []
}

logs = {}
callbacks.on_train_begin(logs=logs)

for epoch in range(num_epochs):
    callbacks.on_epoch_begin(epoch=epoch, logs=logs)

    for x, y in ds_train:
        logs = train_step(x, y)
    history["train_loss"].append(logs["train_loss"].numpy())
    history["train_accuracy"].append(logs["train_accuracy"].numpy())

    for x, y in ds_test:
        logs = test_step(x, y)
    history["test_loss"].append(logs["test_loss"].numpy())
    history["test_accuracy"].append(logs["test_accuracy"].numpy())

    callbacks.on_epoch_end(epoch=epoch, logs=logs)

callbacks.on_train_end(logs=logs)
print("Time taken: %.2fs"%(time.time() - start_time))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
for ax, m in zip(axes, ["loss", "accuracy"]):
    for l in ["train", "test"]:
        g = sns.lineplot(ax=ax, label=l, x=range(num_epochs), y=history[f"{l}_{m}"])
    ax.legend(loc="best")
    ax.set_xlabel(xlabel="epoch")
    ax.set_ylabel(ylabel=f"{m}")
    ax.set_title(label=f"{m} vs. epoch")
fig.tight_layout()
plt.show()